In [81]:
##Importing libraries

import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Question1

In [95]:
###reading the data
january = pd.read_parquet("fhv_tripdata_2021-01.parquet")
february = pd.read_parquet("fhv_tripdata_2021-02.parquet")

In [3]:
print(f"The total number of record in january is {len(january)}")

The total number of record in january is 1154112


### Question 2

In [4]:
january.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [5]:
###getting duration of the ride
january['dropOff_datetime'] = pd.to_datetime(january['dropOff_datetime'])
january['pickup_datetime'] = pd.to_datetime(january['pickup_datetime'])


january["duration"]  = january['dropOff_datetime'] - january['pickup_datetime']

In [27]:
january['duration'] = january['duration'].apply(lambda x : x.total_seconds()/60)

In [29]:
print("The average duration is: ", january['duration'].mean())

The average duration is:  19.1672240937939


In [44]:
###dataframe with minutes greater than 60


print("Number of records to be dropped are: ",len(january[january["duration"]>60]))
january = january[january["duration"]<=60]

Number of records to be dropped are:  29262


## Question3

In [47]:
##Fraction of missing value present

total = january['PUlocationID'].isna().sum()
print("The fraction of the missing value present in pickup location ID", total/len(january) * 100)

The fraction of the missing value present in pickup location ID 83.18504689514158


In [49]:
##filling the missing values with "-1"
january.fillna(-1,inplace=True)

In [51]:
january.isna().sum()

dispatching_base_num      0
pickup_datetime           0
dropOff_datetime          0
PUlocationID              0
DOlocationID              0
SR_Flag                   0
Affiliated_base_number    0
duration                  0
dtype: int64

### Question4

In [69]:
### converting DOlocationID  and PUlocationID to strings

categorical  = ["PUlocationID", "DOlocationID"]
january[categorical] = january[categorical].astype('str')

In [70]:
january.head(6)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,-1,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,-1,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,-1,B00037,9.050000
6,B00037,2021-01-01 00:18:12,2021-01-01 00:30:04,-1.0,91.0,-1,B00037,11.866667


In [71]:
train_dict = january[categorical].to_dict(orient='records')

In [72]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dict)

In [73]:
print("The dimensionality of the matrix is", X_train.shape[1])

The dimensionality of the matrix is 525


### Question 5

In [78]:
target = january['duration'].values

In [79]:
lr = LinearRegression()

lr.fit(X_train,target)

LinearRegression()

In [82]:
print("The mean squared error on the training data is",np.sqrt(mean_squared_error(lr.predict(X_train),target)))

The mean squared error on the training data is 10.629704061671607


### Question 6

In [87]:
def obtain_duration(data) ->pd.DataFrame:
    data["duration"]  = data['dropOff_datetime'] - data['pickup_datetime']
    data['duration'] = data['duration'].apply(lambda x : x.total_seconds()/60)
    return data

In [88]:
def convert_to_datetime(data) ->pd.DataFrame:
    data['dropOff_datetime'] = pd.to_datetime(data['dropOff_datetime'])
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
    return data

In [89]:
def dropping_and_filling_value(data)->pd.DataFrame:
    data = data[data["duration"]<=60]
    data.fillna(-1,inplace=True)
    return data

In [91]:
def return_data(data):
    data[categorical] = data[categorical].astype('str')
    test_dict = data[categorical].to_dict(orient='records')
    test_dict = dv.transform(test_dict)
    return test_dict, data['duration'].values

In [96]:
february = obtain_duration(february)
february = convert_to_datetime(february)
february = dropping_and_filling_value(february)
X_test, target = return_data(february)

/tmp/ipykernel_58/3920698523.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(-1,inplace=True)
/tmp/ipykernel_58/4249647402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[categorical] = data[categorical].astype('str')


In [97]:
x_target = lr.predict(X_test)

In [98]:
print("The test mean squared error on the training data is",np.sqrt(mean_squared_error(x_target,target)))

The test mean squared error on the training data is 11.115605155428213
